In [3]:
pip install faiss-cpu

In [4]:
pip install langchain_community

In [5]:
import os
import numpy as np
import faiss
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

# Chemins des fichiers
file_paths = [
    '/content/drive/MyDrive/Processed_Data/Arcep_pdf.txt',
    '/content/drive/MyDrive/Processed_Data/Arcep_txt.txt',
    '/content/drive/MyDrive/Processed_Data/DemarchesMTFP_txt.txt',
    '/content/drive/MyDrive/Processed_Data/FAQ-CGI_txt.txt',
    '/content/drive/MyDrive/Processed_Data/Immigration_txt.txt',
    '/content/drive/MyDrive/Processed_Data/VIDEO-CGI_txt.txt',
    '/content/drive/MyDrive/Processed_Data/code decrets lois_pdf.txt',
    '/content/drive/MyDrive/Processed_Data/codes-decrets-lois_pdf.txt',
    '/content/drive/MyDrive/Processed_Data/service_public_csv.txt',
    '/content/drive/MyDrive/Processed_Data/service_public_html.txt'
]

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
# Chargement des documents
documents = []
for path_file in file_paths:
    if os.path.exists(path_file):
        loader = TextLoader(path_file)
        documents.extend(loader.load())
    else:
        print(f"Le fichier {path_file} n'existe pas.")


In [7]:
# Découpage des documents en morceaux
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=0)
split_docs = text_splitter.split_documents(documents)

In [8]:
# Charger le modèle Sentence-Transformer pour les embeddings
model = SentenceTransformer('intfloat/multilingual-e5-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
# Extraire le texte des documents découpés
texts = [doc.page_content for doc in split_docs]


In [10]:
# Générer les embeddings
embeddings = model.encode(texts, show_progress_bar=True)

# Convertir les embeddings en un array numpy de type float32 pour FAISS
embeddings_array = np.array(embeddings, dtype=np.float32)

Batches:   0%|          | 0/289 [00:00<?, ?it/s]

In [11]:
# Initialiser l'index FAISS
index = faiss.IndexFlatL2(embeddings_array.shape[1])
index.add(embeddings_array)

In [12]:
# Sauvegarder l'index FAISS sur disque pour une utilisation future
faiss.write_index(index, "/content/faiss_index.index")
print(f"Index FAISS sauvegardé. Nombre de vecteurs indexés : {index.ntotal}")


Index FAISS sauvegardé. Nombre de vecteurs indexés : 9227


In [13]:
# Fonction pour rechercher un document dans l'index
def search_in_index(query, k=5):
    query_embedding = model.encode([query], show_progress_bar=False)
    Similarite, indices = index.search(np.array(query_embedding).astype(np.float32), k)
    return Similarite, indices

In [14]:
# Charger l'index FAISS depuis le disque
index = faiss.read_index("/content/faiss_index.index")

In [15]:
# Exemple de recherche
query ='comment faire pour avoir un acte de naissance'
Similarite, indices = search_in_index(query)

# Affichage des résultats de la recherche
print(f"Résultats pour la requête : '{query}'")
for i, idx in enumerate(indices[0]):
    print(f"  chunk {i+1}: {split_docs[idx]}")  # Affichage des 300 premiers caractères
    print(f"  similarite: {Similarite[0][i]}")
    print()

Résultats pour la requête : 'comment faire pour avoir un acte de naissance'
  chunk 1: page_content='"Est-il obligatoire d’enregistrer un acte ?' metadata={'source': '/content/drive/MyDrive/Processed_Data/FAQ-CGI_txt.txt'}
  similarite: 0.2698705792427063

  chunk 2: page_content='Catégorie: Agence Nationale d’Identification des Personnes.\nTitre: Délivrance d'acte de naissance sécurisé (Part 1).\nDélivrance d'acte de naissance sécurisé.  Acte de naissance PRAN/PEDEC  Acte de naissance copie intégrale  Acte de naissance par transcription. Durée estimée: 48h. Coût de service: 1000 FCFA. Site Internet: https://eservices.anip.bj. Qui peut faire la demande? Tout citoyen de nationalité béninoise. Institution en charge du service: Agence Nationale d’Identification des Personnes https://service-public.bj/public/services/service/PS00393                                          Agence Nationale d’Identification des' metadata={'source': '/content/drive/MyDrive/Processed_Data/service_public_csv.t

In [16]:
!pip install bitsandbytes


In [17]:
faiss.write_index(index, "/content/faiss_index.index")


In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

def load_quantized_model(llm_model: str):
    """
    Charge un modèle quantifié en 4 bits.

    :param llm_model: Le nom du modèle ou le chemin du modèle.
    :return: Le modèle quantifié chargé.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        llm_model,  # Correction ici : utilisation de llm_model
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        config=bnb_config
    )

    return model


In [19]:
# Recherche dans l'index FAISS et génération de réponse
def search_and_generate_response(query, index, documents, model, llm_model, tokenizer, top_k=2):
    """
    Recherche des documents similaires dans l'index FAISS et génère une réponse à partir du modèle LLM.

    :param query: La requête à rechercher dans l'index FAISS.
    :param index: L'index FAISS contenant les embeddings des documents.
    :param documents: Liste des documents à interroger.
    :param model: Le modèle Sentence-Transformer pour générer l'embedding de la requête.
    :param llm_model: Le modèle de langage pour générer la réponse.
    :param tokenizer: Le tokenizer associé au modèle LLM.
    :param top_k: Le nombre de résultats à récupérer.
    :return: La réponse générée.
    """
    start_time = time.time()

    # Étape 1 : Générer un embedding pour la requête
    query_embedding = model.encode([query])
    assert query_embedding.shape[1] == index.d, "La dimension de l'embedding de la requête ne correspond pas à celle de l'index FAISS."
    print(f"Temps pour générer l'embedding : {time.time() - start_time:.2f}s")

    # Étape 2 : Recherche dans FAISS
    start_time = time.time()
    distances, indices = index.search(np.array(query_embedding, dtype=np.float32), k=top_k)
    print(f"Temps pour la recherche FAISS : {time.time() - start_time:.2f}s")

    # Étape 3 : Récupérer les documents correspondants
    retrieved_docs = [documents[i] for i in indices[0]]
    print(f"Documents récupérés : {retrieved_docs}")

    # Étape 4 : Construire le contexte et générer une réponse
    start_time = time.time()
    context = "\n\n".join(retrieved_docs)
    prompt = f"Voici des informations trouvées dans la base de données :\n\n{context}\n\nQuestion : {query}\nRéponse :"

    # Tokenisation et génération de la réponse
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=150,
        pad_token_id=tokenizer.eos_token_id,  # Définit un token de padding
    )
    print(f"Temps pour la génération LLM : {time.time() - start_time:.2f}s")

    # Décodage de la réponse générée
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [20]:
# Mettre à jour bitsandbytes
!pip install -U bitsandbytes



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Charger un modèle standard
llm_model = AutoModelForCausalLM.from_pretrained('filipealmeida/Mistral-7B-Instruct-v0.1-sharded')

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained('filipealmeida/Mistral-7B-Instruct-v0.1-sharded')

# Exemple d'utilisation
query = "Comment puis-je immigrer en France ?"
index = faiss.read_index("/content/faiss_index.index")  # Votre index FAISS
documents = ["Document 1", "Document 2", "Document 3"]  # Exemple de documents

# Recherche et génération de la réponse
response = search_and_generate_response(query, index, documents, model, llm_model, tokenizer)
print(f"Réponse générée : {response}")


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00004-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00005-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [24]:
!pip install accelerate


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator

# Initialiser l'accélérateur
accelerator = Accelerator()

# Charger un modèle lourd sans quantification 4 bits
model_name = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Préparer le modèle et le tokenizer avec Accelerator
model = accelerator.prepare(model)

# Exemple d'utilisation
query = "Comment puis-je immigrer en France ?"
input_ids = tokenizer(query, return_tensors="pt").input_ids

# Générer une réponse
outputs = model.generate(input_ids, max_length=150, num_return_sequences=1)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Réponse générée : {response}")


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]